In [1]:
# Importing all the necessary packages

import pandas as pd
import numpy as np
import os

In [2]:
directory = os.path.dirname(os.getcwd()) 
path=directory+'/Recommendation_Systems/Data'
#path

In [3]:
item_cols = ['item_id', 'movie_title','release_date','video','release_date','IMDb_URL','unknown', 'Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
items = pd.read_csv(path+'/u.item', sep='|',names=item_cols,encoding='latin-1')

items.head(2)

,item_id,movie_title,release_date,video,release_date.1,IMDb_URL,unknown,Action,Adventure,Animation,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,NaN
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,NaN


In [4]:
user_cols = ['user_id', 'item_id', 'rating', 'timestamp']
user = pd.read_csv(path+'/u.data', sep='\t', names=user_cols,encoding='latin-1')

user.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
dataset = pd.merge(items[['item_id','movie_title']],user[['user_id','rating','item_id']], on='item_id' )
dataset.head()

,item_id,movie_title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [6]:
#dataset[['item_id']].count().drop_duplicates()

In [7]:
#print(dataset.item_id.unique().shape[0])
#print(dataset.user_id.unique().shape[0])

In [8]:
#dataset.movie_title.unique().shape[0]

In [9]:
f = lambda dataset: dataset.sort('item_id', ascending=False)
a=dataset[['item_id','movie_title']].groupby('movie_title').apply(f).reset_index(0,drop=True).drop_duplicates().sort('item_id', ascending=True)

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [10]:
a.head()

,item_id,movie_title
0,1,Toy Story (1995)
452,2,GoldenEye (1995)
583,3,Four Rooms (1995)
673,4,Get Shorty (1995)
882,5,Copycat (1995)


In [47]:
X = dataset.pivot(index ='user_id',columns='item_id',values='rating').fillna(0)
X.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# Pearson correlation method
def pearson_corr(x,y):
    x_cord=x-x.mean()
    #print(x,x_cord)
    y_cord=y-y.mean()
    #print(y,y_cord)
    
    num = np.sum(x_cord * y_cord)
    den = np.sqrt(np.sum(x_cord**2) * np.sum(y_cord**2))
    return num/den

In [50]:
def recommendations(name,count):
    id = int(a[a['movie_title']==name]['item_id'])
    r=[]
    w={}
    for e in X.columns:
        if e==id:
            continue
        c=pearson_corr(X[id],X[e])
        #if np.isnan(c):
            #continue
        #else:
        r.append((e,c))
    r.sort(key=lambda tup: tup[1],reverse=True)
    for t in range(count):
        q = r[t][0]
        title = (a[a['item_id']==q]['movie_title']).to_string(index = False)
        w[title] = r[t][1]
    return(w)

In [51]:
recommendations("Copycat (1995)",5)

{'Candyman (1992)': 0.39363004951088576,
 'Cape Fear (1991)': 0.49103364330970312,
 'Natural Born Killers (1994)': 0.38569686559508543,
 'Nightmare on Elm Street, A (1984)': 0.41940513567093241,
 'Outbreak (1995)': 0.41825547992283552}